In [1]:
from datasets import load_dataset
from tqdm import tqdm
from knowledge_propagation.utils import vars, io
from scipy.stats import describe

/home/zliu/miniconda3/envs/cpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rc = load_dataset("/home/zliu/shared_resources/datasets/trivia_qa", 'rc.web')

In [3]:
non_question_ends = []
multiple_question_mark = []
for split in ["train", "validation", "test"][:]:
    clean_sft_data = []
    
    for datum in tqdm(rc[split],):
        q = datum["question"]
        a = datum["answer"]["value"]
        if q[0] == q[-1] == "\"" or q[0] == q[-1] == "\'":
            q = q[1:-1]
        
        if not q.endswith("?"):
            non_question_ends.append(q)
            continue
        
            
        t = f"{q}{a}" if a[0] == " " else f"{q} {a}"
        if t.count("?") != 1:
            multiple_question_mark.append(q)
            continue
        clean_sft_data.append(
            {
                "question": q,
                "answer": a,
                "text": t,
            }
        )
    print("Split =", split)
    print("# multiple_question_mark ", len(multiple_question_mark))
    print("# not ending in `?`", len(non_question_ends))
    print("# clean instances", len(clean_sft_data))
    io.dump_jsonlines(clean_sft_data, f"{vars.DATA_DIR}/trivia_qa_sft/{split[:5]}.jsonl") 
    

100%|██████████| 76496/76496 [00:17<00:00, 4285.37it/s]


Split = train
# multiple_question_mark  386
# not ending in `?` 4111
# clean instances 71999


100%|██████████| 9951/9951 [00:02<00:00, 4379.83it/s]


Split = validation
# multiple_question_mark  440
# not ending in `?` 4675
# clean instances 9333


100%|██████████| 9509/9509 [00:02<00:00, 4703.56it/s]


Split = test
# multiple_question_mark  499
# not ending in `?` 5214
# clean instances 8911


In [58]:
print(len(multiple_question_mark))

297


In [ ]:
# {"question": , "answer": , "text": }
rc

DatasetDict({
    train: Dataset({
        features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
        num_rows: 138384
    })
    validation: Dataset({
        features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
        num_rows: 17944
    })
    test: Dataset({
        features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
        num_rows: 17210
    })
})

In [67]:
describe([len(vars.GPT_4_TOKENIZER(x["text"])) for x in io.load_jsonlines(f"{vars.DATA_DIR}/trivia_qa_sft/train.jsonl")])

DescribeResult(nobs=72220, minmax=(np.int64(7), np.int64(137)), mean=np.float64(21.41464968152866), variance=np.float64(84.33244576409174), skewness=np.float64(1.8022750710654165), kurtosis=np.float64(6.216870062081904))

0

True